In [1]:
import pandas as pd
import numpy as np

In [2]:
raw_df = pd.read_csv('craigslist_cleaned.csv')
raw_backup = raw_df.copy()

In [ ]:
# sampled_df = raw_df.sample(50)
# sampled_df.to_csv('craigslist_cleaned_sample_50.csv')

In [3]:
# raw_df = raw_backup.copy()

In [3]:
raw_df.head()

,region,price,year,manufacturer,model,condition,cylinders,fuel,odometer,title_status,...,drive,type,paint_color,state,lat,long,posting_date,posting_date_formatted,quarter,id
0,hudson valley,18997,2013,dodge,durango,excellent,6 cylinders,gas,96475,clean,...,4wd,suv,black,ny,42.598179,-73.708295,2021-04-04 07:00:00,2021-04-04 07:00:25,Y2021Q2,0
1,hudson valley,51997,2017,chevrolet,silverado 2500hd,excellent,8 cylinders,diesel,73231,clean,...,4wd,pickup,black,ny,42.598179,-73.708295,2021-04-04 07:00:00,2021-04-04 07:00:51,Y2021Q2,1
2,hudson valley,28997,2016,toyota,tacoma,excellent,6 cylinders,gas,93407,clean,...,4wd,pickup,not_available,ny,42.598179,-73.708295,2021-04-04 07:01:00,2021-04-04 07:01:16,Y2021Q2,2
3,kenosha-racine,119,2013,dodge,dart,excellent,4 cylinders,gas,71800,not_available,...,not_available,sedan,not_available,wi,42.595596,-87.879408,2021-04-04 07:03:00,2021-04-04 07:03:15,Y2021Q2,3
4,richmond,18937,2011,missing,forde350,not_available,missing,gas,40000,clean,...,not_available,not_available,not_available,va,33.820600,-84.354900,2021-04-04 07:08:00,2021-04-04 07:08:37,Y2021Q2,4


In [4]:
def coerce_types(raw_df: pd.DataFrame) -> pd.DataFrame:
    df = raw_df.copy()

    # Columns to force as nullable integers
    for col in ["price", "year", "odometer"]:
        if col in df.columns:
            df[col] = pd.to_numeric(df[col], errors="coerce").astype("Int64")

    # posting_date → datetime64[ns], drop tz to be naive, truncate to minutes
    if "posting_date" in df.columns:
        dt = pd.to_datetime(df["posting_date"], errors="coerce", utc=True)
        # Convert to UTC clock time, drop timezone, floor to minute
        df["posting_date"] = dt.dt.tz_convert("UTC").dt.tz_localize(None).dt.floor("min")

    # Everything else as strings
    keep_as_str = [
        "region","manufacturer","model","condition","cylinders","fuel",
        "title_status","transmission","drive","type","paint_color",
        "state","lat","long"
    ]
    for col in keep_as_str:
        if col in df.columns:
            df[col] = df[col].astype("string")

    return df

# Example:
raw_df = coerce_types(raw_df)
raw_df.dtypes  # to verify


region                            string
price                              Int64
year                               Int64
manufacturer                      string
model                             string
condition                         string
cylinders                         string
fuel                              string
odometer                           Int64
title_status                      string
transmission                      string
drive                             string
type                              string
paint_color                       string
state                             string
lat                               string
long                              string
posting_date              datetime64[ns]
posting_date_formatted            object
quarter                           object
id                                 int64
dtype: object

In [5]:
raw_df.head()

,region,price,year,manufacturer,model,condition,cylinders,fuel,odometer,title_status,...,drive,type,paint_color,state,lat,long,posting_date,posting_date_formatted,quarter,id
0,hudson valley,18997,2013,dodge,durango,excellent,6 cylinders,gas,96475,clean,...,4wd,suv,black,ny,42.598179,-73.708295,2021-04-04 07:00:00,2021-04-04 07:00:25,Y2021Q2,0
1,hudson valley,51997,2017,chevrolet,silverado 2500hd,excellent,8 cylinders,diesel,73231,clean,...,4wd,pickup,black,ny,42.598179,-73.708295,2021-04-04 07:00:00,2021-04-04 07:00:51,Y2021Q2,1
2,hudson valley,28997,2016,toyota,tacoma,excellent,6 cylinders,gas,93407,clean,...,4wd,pickup,not_available,ny,42.598179,-73.708295,2021-04-04 07:01:00,2021-04-04 07:01:16,Y2021Q2,2
3,kenosha-racine,119,2013,dodge,dart,excellent,4 cylinders,gas,71800,not_available,...,not_available,sedan,not_available,wi,42.595596,-87.879408,2021-04-04 07:03:00,2021-04-04 07:03:15,Y2021Q2,3
4,richmond,18937,2011,missing,forde350,not_available,missing,gas,40000,clean,...,not_available,not_available,not_available,va,33.8206,-84.3549,2021-04-04 07:08:00,2021-04-04 07:08:37,Y2021Q2,4


In [6]:
import pandas as pd

def categorical_overview(df: pd.DataFrame, categorical_cols: list[str]) -> pd.DataFrame:
    """
    Build a summary table for categorical features.
    - n_unique: distinct non-missing values.
    - n_missing: number of missing values.
    - value_counts: dict of counts including a '__NA__' bucket for missing.
    """
    rows = []
    for col in categorical_cols:
        if col not in df.columns:
            continue

        s = df[col].astype("string")
        n_unique = s.nunique(dropna=True)
        n_missing = int(s.isna().sum())

        vc = s.value_counts(dropna=False)
        idx = vc.index.to_series().astype(object).where(vc.index.notna(), "__NA__").astype(str)
        value_counts_dict = dict(zip(idx, vc.tolist()))

        rows.append({
            "feature": col,
            "n_unique": n_unique,
            "n_missing": n_missing,
            "value_counts": value_counts_dict
        })

    out = pd.DataFrame(rows).sort_values(["n_unique", "feature"], ascending=[False, True]).reset_index(drop=True)
    return out

# Your categorical feature list (per your schema)
categorical_cols = [
    "region", "manufacturer", "model", "condition", "cylinders", "fuel",
    "title_status", "transmission", "drive", "type", "paint_color", "state",
]

# Example usage:
overview_df = categorical_overview(raw_df, categorical_cols)
overview_df


,feature,n_unique,n_missing,value_counts
0,model,27920,0,"{'f-150': 8035, 'silverado 1500': 5135, '1500'..."
1,region,404,0,"{'columbus': 3529, 'jacksonville': 3456, 'sout..."
2,state,51,0,"{'ca': 49507, 'fl': 27818, 'tx': 22169, 'ny': ..."
3,manufacturer,42,0,"{'ford': 69527, 'chevrolet': 53603, 'toyota': ..."
4,type,14,0,"{'not_available': 88945, 'sedan': 86560, 'suv'..."
5,paint_color,13,0,"{'not_available': 125710, 'white': 77712, 'bla..."
6,cylinders,9,0,"{'6 cylinders': 139170, '4 cylinders': 128809,..."
7,condition,7,0,"{'not_available': 168906, 'good': 119854, 'exc..."
8,title_status,7,0,"{'clean': 396193, 'not_available': 8107, 'rebu..."
9,fuel,6,0,"{'gas': 350247, 'other': 29785, 'diesel': 2759..."


In [7]:
import pandas as pd
import math

def numeric_describe_table(df: pd.DataFrame, numeric_cols: list[str]) -> pd.DataFrame:
    """
    Returns a DataFrame where each row is a feature
    and columns are the standard .describe() stats + n_missing.
    """
    rows = []
    for col in numeric_cols:
        if col not in df.columns:
            continue
        s = pd.to_numeric(df[col], errors="coerce")
        d = s.describe(percentiles=[0.25, 0.5, 0.75])  # count, mean, std, min, 25%, 50%, 75%, max
        n_missing = int(s.isna().sum())

        rows.append(
            {
                "feature": col,
                "count": float(d.get("count", math.nan)),  # non-missing count
                "n_missing": n_missing,                   # number of NaNs
                "mean": float(d.get("mean", math.nan)),
                "std": float(d.get("std", math.nan)),
                "min": float(d.get("min", math.nan)),
                "25%": float(d.get("25%", math.nan)),
                "50%": float(d.get("50%", math.nan)),
                "75%": float(d.get("75%", math.nan)),
                "max": float(d.get("max", math.nan)),
            }
        )
    out = pd.DataFrame(rows).set_index("feature")
    return out

# Your numeric features per schema
numeric_cols = ["price", "year", "odometer"]   # core numeric
geo_cols = ["lat", "long"]                     # optional numeric (geo)

# Examples:
core_stats = numeric_describe_table(raw_df, numeric_cols)
all_numeric_stats = numeric_describe_table(raw_df, numeric_cols + geo_cols)
display(all_numeric_stats)


,count,n_missing,mean,std,min,25%,50%,75%,max
feature,,,,,,,,,
price,417358.0,0,17057.140534,14195.020652,0.000000,5950.0000,13950.0000,25995.000000,74999.000000
year,417358.0,0,2011.422275,8.994576,1900.000000,2008.0000,2014.0000,2017.000000,2022.000000
odometer,417358.0,0,90885.551939,61601.351931,0.000000,38561.0000,85548.0000,132000.000000,343862.000000
lat,417358.0,0,38.509633,5.780188,-81.838232,34.7789,39.1501,42.348459,82.252826
long,417358.0,0,-94.568086,18.227199,-159.827728,-111.8000,-88.4326,-81.007873,167.629911


In [8]:
import pandas as pd

def check_posting_date_missing(df: pd.DataFrame, col: str = "posting_date"):
    # Coerce to datetime (keeps existing tz, treats bad values as NaT)
    s_dt = pd.to_datetime(df[col], errors="coerce", utc=True)

    # Missing mask (NaN/NaT both count as missing)
    missing_mask = s_dt.isna()
    n_missing = int(missing_mask.sum())
    n_total = int(len(s_dt))

    print(f"[posting_date] missing: {n_missing} / {n_total} "
          f"({(n_missing / n_total * 100):.2f}%)")

    if n_missing:
        # Show a few offending raw values for debugging
        bad_samples = (
            df.loc[missing_mask, col]
              .astype("string")
              .value_counts(dropna=False)
              .head(10)
        )
        print("\nTop problematic raw values (up to 10):")
        print(bad_samples)

        # Optional: return indices of rows with missing dates
        return df.index[missing_mask]
    return pd.Index([])

# Example:
bad_idx = check_posting_date_missing(raw_df)
raw_df.loc[bad_idx].head()


[posting_date] missing: 0 / 417358 (0.00%)


,region,price,year,manufacturer,model,condition,cylinders,fuel,odometer,title_status,...,drive,type,paint_color,state,lat,long,posting_date,posting_date_formatted,quarter,id


In [9]:
# raw_df.to_csv('datasets\craigslist_cleaned.csv', index=False)